### Imports

In [ ]:
import torch
import torch.nn as nn
from torch import nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import SequentialLR, LinearLR, CosineAnnealingLR
import torch.distributed as dist # Maybe in the future we need multiple gpu's for training for now this is just a starting point
from torch import Tensor
from torchvision import transforms

from tqdm import tqdm

import numpy as np


import sys
import time
import math
from typing import List
import random
import os
import datetime
import subprocess
from collections import defaultdict, deque


from PIL import Image, ImageFilter, ImageOps

### Utils

In [ ]:
def _no_grad_trunc_normal_(tensor, mean, std, a, b):
    # Cut & paste from PyTorch official master until it's in a few official releases - RW
    # Method based on https://people.sc.fsu.edu/~jburkardt/presentations/truncated_normal.pdf
    def norm_cdf(x):
        # Computes standard normal cumulative distribution function
        return (1. + math.erf(x / math.sqrt(2.))) / 2.

    if (mean < a - 2 * std) or (mean > b + 2 * std):
        warnings.warn("mean is more than 2 std from [a, b] in nn.init.trunc_normal_. "
                      "The distribution of values may be incorrect.",
                      stacklevel=2)

    with torch.no_grad():
        # Values are generated by using a truncated uniform distribution and
        # then using the inverse CDF for the normal distribution.
        # Get upper and lower cdf values
        l = norm_cdf((a - mean) / std)
        u = norm_cdf((b - mean) / std)

        # Uniformly fill tensor with values from [l, u], then translate to
        # [2l-1, 2u-1].
        tensor.uniform_(2 * l - 1, 2 * u - 1)

        # Use inverse cdf transform for normal distribution to get truncated
        # standard normal
        tensor.erfinv_()

        # Transform to proper mean, std
        tensor.mul_(std * math.sqrt(2.))
        tensor.add_(mean)

        # Clamp to ensure it's in the proper range
        tensor.clamp_(min=a, max=b)
        return tensor


def trunc_normal_(tensor, mean=0., std=1., a=-2., b=2.):
    # type: (Tensor, float, float, float, float) -> Tensor
    return _no_grad_trunc_normal_(tensor, mean, std, a, b)


class GaussianBlur(object):
    """
    Apply Gaussian Blur to the PIL image.
    """
    def __init__(self, p=0.5, radius_min=0.1, radius_max=2.):
        self.prob = p
        self.radius_min = radius_min
        self.radius_max = radius_max

    def __call__(self, img):
        do_it = random.random() <= self.prob
        if not do_it:
            return img

        return img.filter(
            ImageFilter.GaussianBlur(
                radius=random.uniform(self.radius_min, self.radius_max)
            )
        )


class Solarization(object):
    """
    Apply Solarisation to the PIL image.
    """
    def __init__(self, p):
        self.p = p

    def __call__(self, img):
        if random.random() < self.p:
            return ImageOps.solarize(img)
        else:
            return img

### Seed

In [ ]:
def set_device():
  """
  Set the device. CUDA if available, CPU otherwise

  Args:
    None

  Returns:
    Nothing
  """
  device = "cuda" if torch.cuda.is_available() else "cpu"
  if device != "cuda":
    print("WARNING: For this notebook to perform best, "
        "if possible, in the menu under `Runtime` -> "
        "`Change runtime type.`  select `GPU` ")
  else:
    print("GPU is enabled in this notebook.")

  return device

def set_seed(seed=None, seed_torch=True):
  """
  Function that controls randomness.
  NumPy and random modules must be imported.

  Args:
    seed : Integer
      A non-negative integer that defines the random state. Default is `None`.
    seed_torch : Boolean
      If `True` sets the random seed for pytorch tensors, so pytorch module
      must be imported. Default is `True`.

  Returns:
    Nothing.
  """
  if seed is None:
    seed = np.random.choice(2 ** 32)
  random.seed(seed)
  np.random.seed(seed)
  if seed_torch:
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

  print(f'Random seed {seed} has been set.')

def seed_worker(worker_id):
  """
  DataLoader will reseed workers following randomness in
  multi-process data loading algorithm.

  Args:
    worker_id: integer
      ID of subprocess to seed. 0 means that
      the data will be loaded in the main process
      Refer: https://pytorch.org/docs/stable/data.html#data-loading-randomness for more details

  Returns:
    Nothing
  """
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)

SEED = 2021
set_seed(seed=SEED)
DEVICE = set_device()

Random seed 2021 has been set.
GPU is enabled in this notebook.


more or less the same as original one i didn't changed much just added couple of things here and there

### Arguments

In [ ]:
# see https://github.com/facebookresearch/dino/blob/main/main_dino.py get_args_parser() for explanations

class args_class:
  def __init__(self):
    # Model Parameters
    self.MOMENTUM_TEACHER = 0.996     #default 0.996
    self.OUT_DIM = 65536 #default 65536
    self.NORM_LAST_LAYER = True
    self.USE_BN_IN_HEAD = False     #default False

    # DINO LOSS Paramters
    self.CENTER_MOMENTUM = 0.9 # default 0.9

    # Temperature Student Parameters
    self.STUDENT_TEMP = 0.1 # default 0.1

    # Temperature Teacher Parameters
    self.WARMUP_TEACHER_TEMP = 0.04   #default 0.04
    self.TEACHER_TEMP = 0.04   #default 0.04
    self.WARMUP_TEACHER_TEMP_EPOCHS = 30 #default 30, but erroneously default 0 in the DINO paper?

    # Training / Optimizations Parameters
    self.USE_FP16 = True #default True
    self.WEIGHT_DECAY = 0.04 #default 0.04
    self.WEIGHT_DECAY_END = 0.4 # default 0.4
    self.CLIP_GRAD = 3.0 #default 3.0
    self.BATCH_SIZE_PER_GPU = 64 #default 64
    self.EPOCHS = 100 #default 100
    self.FREEZE_LAST_LAYER = 1 #default 1
    self.LR = 0.0005 #default 0.0005
    self.WARMUP_EPOCHS = 10 #default 10
    self.MIN_LR = 1e-6 #default 1e-6
    self.optimizer = 'adamw' # default adamw    TODO:Could be constructed here? not sure.
    self.DROP_PATH_RATE = 0.1 #default 0.1

    # Multi-Crop Parameters
    self.GLOBAL_CROPS_NUMBER = 2 #default 2
    self.GLOBAL_CROPS_SCALE = (0.4, 0.1) # default (0.4, 0.1)
    self.LOCAL_CROPS_NUMBER = 8 #default 8
    self.LOCAL_CROPS_SCALE = (0.05, 0.4) #default (0.05, 0.4)

    #Misc
    self.num_works = 10 # default 10
    # TODO: imgnet directory, how to store weights?

args = args_class()

### Architecture, DINOHead

In [ ]:
class DINOHead(nn.Module):
  def __init__(self, in_dim, out_dim, use_bn=False, norm_last_layer=True, nlayers=3, hidden_dim=2048, bottleneck_dim=256):
    super().__init__()
    nlayers = max(nlayers, 1)
    if nlayers == 1:
      self.mlp = nn.Linear(in_dim, bottleneck_dim)
    else:
      layers = [nn.Linear(in_dim, hidden_dim)]
      if use_bn:
        layers.append(nn.BatchNorm1d(hidden_dim))
      layers.append(nn.GELU())
      for _ in range(nlayers - 2):
        layers.append(nn.Linear(hidden_dim, hidden_dim))
        if use_bn:
          layers.append(nn.BatchNorm1d(hidden_dim))
        layers.append(nn.GELU())
      layers.append(nn.Linear(hidden_dim, bottleneck_dim))
      self.mlp = nn.Sequential(*layers)
    self.apply(self._init_weights)
    self.last_layer = nn.utils.weight_norm(nn.Linear(bottleneck_dim, out_dim, bias=False))
    self.last_layer.weight_g.data.fill_(1)
    if norm_last_layer:
      self.last_layer.weight_g.requires_grad = False

    def _init_weights(self, m):
      if isinstance(m, nn.Linear):
        trunc_normal_(m.weight, std=.02)
        if isinstance(m, nn.Linear) and m.bias is not None:
          nn.init.constant_(m.bias, 0)

    def forward(self, x):
      x = self.mlp(x)
      x = nn.functional.normalize(x, dim=-1, p=2)
      x = self.last_layer(x)
      return x

### ResNet18

In [ ]:
def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )

def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

class ResidualConvBlock(nn.Module):

    """
    Our copy of BasicBlock from torchvision.models.resnet

    """
    # To increease the final number of output channels
    expansion: int = 1

    def __init__(
        self,
        in_channels,
        out_channels:int,
        stride:int = 1,
        downsample = None,
        groups:int = 1,
        base_width:int = 64,
        dilation:int = 1,
        norm_layer = None

        ) -> None:
        super().__init__()

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        # Groups are != 1 when we use different filters for different channels of the image
        if groups != 1 or base_width != 64:
            raise ValueError("BasicBlock only supports groups=1 and base_width=64")

        # Dilation means adding "empty" pixels in the kernel, effectively making it bigger
        # Dilation == 1 means no dilation
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")

        # Note: names bn1, bn2 etc are kept even if norm_layer is not BatchNorm2d
        self.conv1 = conv3x3(in_channels, out_channels, stride) # conv3x3 is standard 3x3 conv
        self.bn1 = norm_layer(out_channels) # Normalise each feature map across spatial and batch dimension
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = norm_layer(out_channels)

        self.downsample = downsample
        self.stride = stride

    def forward(self, x):

        identity = x  # Store original input

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)

        if self.downsample is not None:
            identity = self.downsample(identity)

        x += identity

        x = self.relu(x)

        return x


class BottleneckConvBlock(nn.Module):

    """
    For resnet50 and above, insted of simply residual blocks, they used bottleneck blocks.

    They say it was more for computational efficiency than accuracy.


    In the original paper https://arxiv.org/abs/1512.03385 they used stride 1x1
    in the first layer of the bottleneck block.

    Apparently 3x3 is better and is what is implemented in torchvision.models.resnet

    """

    expansion: int = 4 # channels in the output is 4 times the number of channels in the input

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample = None,
        groups: int = 1, # Bottleneck blocks can have parallel convolutions with groups!=1 ( ResNeXt)
        base_width: int = 64, # How many channels in the internal 3x3 conv of each group
        dilation: int = 1,
        norm_layer = None
    ) -> None:
        super().__init__()

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        width = int(inplanes * (base_width / 64.0) ) * groups #(64 was the default val for base_width)

        self.conv1 = conv1x1(inplanes, width) # 1x1 conv to reduce the number of channels
        self.bn1 = norm_layer(width)

        self.conv2 = conv3x3(width, width, stride, groups, dilation) # the actual convolution in the block
        self.bn2 = norm_layer(width)

        self.conv3 = conv1x1(width, planes * self.expansion) # 1x1 conv to increase the number of channels
        self.bn3 = norm_layer(planes * self.expansion)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample # Downsample is used to match the number of channels in the input and output
        self.stride = stride


    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class OurResNet(nn.Module):
    """
    Our copy of ResNet from torchvision.models.resnet

    """

    def __init__(
        self,
        block, # block can be either ResidualConvBlock or Bottleneck class
        layers: List[int], # List of residual blocks in each layer, e.g. [2, 2, 2, 2] for ResNet18
        num_classes: int = 1000,
        zero_init_residual: bool = False, # not implemented Zero-initialize the last BN in each residual branch,
        groups: int = 1, # in Bottleneck
        width_per_group: int = 64, # in Bottleneck
        replace_stride_with_dilation: List[bool] = None, # in Bottleneck
        norm_layer = None
    ) -> None:
        super().__init__()

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1

        # Resnet has 4 layer gropus, only in the last 3 dilation can substitute stride
        if replace_stride_with_dilation is None:
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:

            raise ValueError(
                "replace_stride_with_dilation should be None "
                f"or a 3-element tuple, got {replace_stride_with_dilation}"
            )

        self.groups = groups
        self.base_width = width_per_group

        # Reference for the following is Table 1 of ResNet paper https://arxiv.org/abs/1512.03385

        # First conv is 7x7 in avery resnet, stride 2, padding 3
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)

        # Maxpool is 3x3 with stride 2 and padding 1
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Initialize the layer groups
        # Numbers 64,128,256,512 get multiplyied by the expansion factor of the block (1 for ResidualConvBlock, 4 for BottleneckConvBlock)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # I just copied this zero_init_residual part

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, BottleneckConvBlock) and m.bn3.weight is not None:
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BottleneckConvBlock) and m.bn2.weight is not None:
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    # With this, we can create any of the layer groups in square brackets(x2,x3,x6,x36 etc)
    # of Table 1 of the ResNet paper
    def _make_layer(
        self,
        block, # kind of block, can be either ResidualConvBlock or BottleneckConvBlock class
        planes: int,
        blocks: int, # number of blocks in the layer group
        stride: int = 1,
        dilate: bool = False, # if True, use dilation instead of stride
    ) -> nn.Sequential:

        norm_layer = self._norm_layer
        downsample = None

        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1

        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer))

        # We always have minimum 2 blocks in each layer group
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer
                )
            )

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)

def _resnet(
    block,
    layers: List[int],
    weights,
    progress: bool,
    **kwargs,
) -> OurResNet:

    if weights is not None:
        # _ovewrite_named_param(kwargs, "num_classes", len(weights.meta["categories"]))
        raise NotImplementedError("Weights upload not implemented yet")

    model = OurResNet(block, layers, **kwargs)

    return model

def ourResnet18( progress: bool = True, **kwargs ) -> OurResNet:
    """
    Constructs a ResNet-18 model.
    """
    return _resnet(
        ResidualConvBlock,
        [2, 2, 2, 2],
        weights=None,  # No weights for now
        progress=progress,
        **kwargs
    )

import torchvision

ourRes18 = ourResnet18(num_classes=10).to(DEVICE)
realRes18 = torchvision.models.resnet18(num_classes=10,weights=None).to(DEVICE)

print(f"weights are close? : {torch.allclose(ourRes18.conv1.weight, realRes18.conv1.weight)}")
print(f"If not, dont expect same logits")

# To have the same weights, they need to have the same exact initialization
ourRes18.load_state_dict(realRes18.state_dict())

print(f"weights are close? : {torch.allclose(ourRes18.conv1.weight, realRes18.conv1.weight)}")
print(f"If not, dont expect same logits")




weights are close? : False
If not, dont expect same logits
weights are close? : True
If not, dont expect same logits
Files already downloaded and verified
Files already downloaded and verified


0it [00:00, ?it/s]

batch_id: 0, ourlogits: -0.09006007760763168, reallogits: -0.09006007760763168


### Utils

In [ ]:
def get_dino_schedules(args, n_iterations_per_epoch):
    """
    Creates the learning rate, weight decay, and teacher momentum schedules
    as pre-calculated NumPy arrays, matching the original DINO implementation.

    Args:
        args (args_class): An object containing training hyperparameters.
        n_iterations_per_epoch (int): The number of training steps in one epoch.

    Returns:
        tuple: A tuple containing:
            - lr_schedule (np.array): The learning rate for each iteration.
            - wd_schedule (np.array): The weight decay for each iteration.
            - momentum_schedule (np.array): The teacher momentum for each iteration.
    """
    # Total number of training iterations
    n_iterations = args.EPOCHS * n_iterations_per_epoch

    # Helper function to replicate the original DINO cosine scheduler
    def cosine_scheduler(base_value, final_value, total_iters, warmup_iters=0, start_warmup_value=0):
        warmup_schedule = np.array([])
        if warmup_iters > 0:
            warmup_schedule = np.linspace(start_warmup_value, base_value, warmup_iters)

        iters = np.arange(total_iters - warmup_iters)
        schedule = final_value + 0.5 * (base_value - final_value) * (1 + np.cos(np.pi * iters / len(iters)))

        schedule = np.concatenate((warmup_schedule, schedule))
        assert len(schedule) == total_iters
        return schedule

    # Linear scaling rule from the original paper: base_lr * batch_size / 256
    # Our code is single-GPU, so we use BATCH_SIZE_PER_GPU. For multi-GPU, this would be scaled by world_size.
    base_lr = args.LR * args.BATCH_SIZE_PER_GPU / 256.0

    lr_schedule = cosine_scheduler(
        base_value=base_lr,
        final_value=args.MIN_LR,
        total_iters=n_iterations,
        warmup_iters=args.WARMUP_EPOCHS * n_iterations_per_epoch,
        start_warmup_value=0, # Start warmup from 0
    )

    # No warmup for weight decay
    wd_schedule = cosine_scheduler(
        base_value=args.WEIGHT_DECAY,
        final_value=args.WEIGHT_DECAY_END,
        total_iters=n_iterations,
    )

    # No warmup for momentum
    momentum_schedule = cosine_scheduler(
        base_value=args.MOMENTUM_TEACHER,
        final_value=1.0,
        total_iters=n_iterations,
    )

    return lr_schedule, wd_schedule, momentum_scheduleclass DINOHead(nn.Module):
  def __init__(self, in_dim, out_dim, use_bn=False, norm_last_layer=True, nlayers=3, hidden_dim=2048, bottleneck_dim=256):
    super().__init__()
    nlayers = max(nlayers, 1)
    if nlayers == 1:
      self.mlp = nn.Linear(in_dim, bottleneck_dim)
    else:
      layers = [nn.Linear(in_dim, hidden_dim)]
      if use_bn:
        layers.append(nn.BatchNorm1d(hidden_dim))
      layers.append(nn.GELU())
      for _ in range(nlayers - 2):
        layers.append(nn.Linear(hidden_dim, hidden_dim))
        if use_bn:
          layers.append(nn.BatchNorm1d(hidden_dim))
        layers.append(nn.GELU())
      layers.append(nn.Linear(hidden_dim, bottleneck_dim))
      self.mlp = nn.Sequential(*layers)
    self.apply(self._init_weights)
    self.last_layer = nn.utils.weight_norm(nn.Linear(bottleneck_dim, out_dim, bias=False))
    self.last_layer.weight_g.data.fill_(1)
    if norm_last_layer:
      self.last_layer.weight_g.requires_grad = False

    def _init_weights(self, m):
      if isinstance(m, nn.Linear):
        trunc_normal_(m.weight, std=.02)
        if isinstance(m, nn.Linear) and m.bias is not None:
          nn.init.constant_(m.bias, 0)

    def forward(self, x):
      x = self.mlp(x)
      x = nn.functional.normalize(x, dim=-1, p=2)
      x = self.last_layer(x)
      return x

### Loss, Train one Epoch, Augmentation


In [ ]:
class DINOLoss(nn.Module):
    """
    The loss function encourages a student network to match the output of a momentum teacher network.
    This is a form of self-distillation without labels.

    Args:
        args (args_class): An object containing the necessary hyperparameters.
    """
    def __init__(self, args):
        super().__init__()
        # The student temperature is constant
        self.student_temp = args.STUDENT_TEMP
        # The number of crops is the two global crops plus all local crops
        self.n_crops = args.GLOBAL_CROPS_NUMBER + args.LOCAL_CROPS_NUMBER
        self.center_momentum = args.CENTER_MOMENTUM
        self.register_buffer("center", torch.zeros(1, args.OUT_DIM)) # Initialize a non-trainable tensor and add to the module

        # The teacher temperature is scheduled to warm up from an initial value to a final value.
        # The DINO paper mentions a warmup from 0.04 to 0.07 over 30 epochs.
        teacher_schedule  = torch.linspace(
            args.WARMUP_TEACHER_TEMP,
            args.TEACHER_TEMP,
            args.EPOCHS,
        )
        self.register_buffer("teacher_temp_schedule", teacher_schedule)

    def forward(self, student_output, teacher_output, epoch):
        """
        Calculates the DINO loss.

        The student is trained to match the teacher's output. The teacher's output is centered
        and sharpened. The loss is computed as the cross-entropy between the teacher's and
        student's probability distributions over different views.

        Args:
            student_output (torch.Tensor): The output of the student network for all crops.
            teacher_output (torch.Tensor): The output of the teacher network for global crops.
            epoch (int): The current training epoch, used for the temperature schedule.

        Returns:
            torch.Tensor: The calculated DINO loss.
        """
        # student side
        student_out = student_output / self.student_temp
        student_out = student_out.chunk(self.n_crops)

        # teacher side: center & sharpen
        temp = self.teacher_temp_schedule[epoch]
        teacher_out = F.softmax((teacher_output - self.center) / temp, dim=-1)
        teacher_out = teacher_out.detach().chunk(2)

        total_loss, n_loss_terms = 0.0, 0
        for iq, q in enumerate(teacher_out):
            for v, sv in enumerate(student_out):
                if v == iq:
                    continue
                loss = torch.sum(-q * F.log_softmax(sv, dim=-1), dim=-1)
                total_loss += loss.mean()
                n_loss_terms += 1

        total_loss /= n_loss_terms
        self._update_center(teacher_output)
        return total_loss

    @torch.no_grad()
    def update_center(self, teacher_output):
        """
        Update the center used for teacher output centering.
        This is a momentum update using the global batch statistics.

        This operation is performed in-place and without gradients.
        """
        # In DDP, teacher_output is the output for the local batch on the current process.
        # We need the mean of the outputs across all processes.
        batch_center = torch.mean(teacher_output, dim=0, keepdim=True)

        # synchronize across all processes
        if dist.is_initialized():
            dist.all_reduce(batch_center, op=dist.ReduceOp.SUM)
            batch_center /= dist.get_world_size()

        # momentum update
        self.center = self.center * self.center_momentum + batch_center * (1 - self.center_momentum)


In [ ]:
def train_one_epoch(student, teacher, dino_loss, data_loader,
                    optimizer, lr_schedule, wd_schedule, momentum_schedule,
                    epoch, loss_scaler, args):
    """
    Trains the DINO model for one epoch using bfloat16 mixed-precision.

    Args:
        student (nn.Module): The student network.
        teacher (nn.Module): The teacher network.
        dino_loss (nn.Module): The DINO loss function.
        data_loader (torch.utils.data.DataLoader): The data loader for the training set.
        optimizer (torch.optim.Optimizer): The optimizer for the student network.
        lr_schedule (torch.optim.lr_scheduler._LRScheduler): The learning rate scheduler.
        wd_schedule (torch.optim.lr_scheduler._LRScheduler): The weight decay scheduler.
        momentum_schedule (torch.optim.lr_scheduler._LRScheduler): The momentum scheduler for the teacher.
        epoch (int): The current epoch number.
        loss_scaler (torch.cuda.amp.GradScaler): The gradient scaler for mixed-precision training.
        args (args_class): The command-line arguments.

    Returns:
        float: The average loss for the epoch.
    """
    pbar = tqdm(data_loader)
    train_loss = 0.0

    for it, (images, _) in enumerate(pbar):
        # Update learning rate and weight decay per iteration
        for i, param_group in enumerate(optimizer.param_groups):
            param_group["lr"] = lr_schedule[it]
            if i == 0:  # only the first group is regularized
                param_group["weight_decay"] = wd_schedule[it]

        # Move images to the GPU
        images = [im.cuda(non_blocking=True) for im in images]

        # Forward pass with bfloat16 automatic mixed-precision
        # Note: bfloat16 requires NVIDIA Ampere (e.g. A100) or newer GPUs.
        with torch.cuda.amp.autocast(enabled=args.USE_FP16, dtype=torch.bfloat16):
            teacher_output = teacher(images[:2]) # Global crops only
            student_output = student(images)
            loss = dino_loss(student_output, teacher_output, epoch)

        if not torch.isfinite(loss):
            print(f"Loss is {loss}, stopping training")
            # Save model weights before exiting
            torch.save({
                'epoch': epoch,
                'student_state_dict': student.state_dict(),
                'teacher_state_dict': teacher.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
            }, 'model_checkpoint_infinite_loss.pth')
            print("Model weights saved to model_checkpoint_infinite_loss.pth")
            import sys
            sys.exit(1)

        # Backward pass and student update
        optimizer.zero_grad(set_to_none=True)
        # The GradScaler scales the loss to prevent underflow
        loss_scaler.scale(loss).backward()
        if args.CLIP_GRAD > 0:
            # Unscale gradients before clipping
            loss_scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(student.parameters(), args.CLIP_GRAD)
        loss_scaler.step(optimizer)
        loss_scaler.update()

        # EMA update for the teacher network
        with torch.no_grad():
            m = momentum_schedule[it]
            for param_q, param_k in zip(student.parameters(), teacher.parameters()):
                param_k.data.mul_(m).add_((1 - m) * param_q.detach().data)

        # Logging
        torch.cuda.synchronize()
        current_lr = optimizer.param_groups[0]['lr']
        train_loss += loss.item()
        pbar.set_description(f"Epoch {epoch} | Loss: {loss.item():.4f} | LR: {current_lr:.6f}")

    return train_loss / len(data_loader)

In [ ]:
class CustomGaussianBlur(object):
    def __init__(self, strength=1.0):
        self.blur = GaussianBlur(p=1.0 if strength > 0.5 else 0.1)

    def __call__(self, img):
        return self.blur(img)


class CustomSolarisation(object):
    def __init__(self, p=0.2):
        self.solar = Solarization(p=p)

    def __call__(self, img):
        return self.solar(img)


class DataAugmentationDINO(object):
    def __init__(self, global_crops_scale, local_crops_scale, local_crops_number):

        flip_and_color_jitter = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomApply(
                [transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1)],
                p=0.8
            ),
            transforms.RandomGrayscale(p=0.2),
        ])

        normalize = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])

        # Global crop 1: strong blur, no solarisation
        self.global_transfo1 = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=global_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            CustomGaussianBlur(strength=1.0),
            normalize,
        ])

        # Global crop 2: weak blur, solarisation
        self.global_transfo2 = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=global_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            CustomGaussianBlur(strength=0.1),
            CustomSolarisation(p=0.2),
            normalize,
        ])

        # Local crop transformation: moderate blur
        self.local_crops_number = local_crops_number
        self.local_transfo = transforms.Compose([
            transforms.RandomResizedCrop(96, scale=local_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            CustomGaussianBlur(strength=0.5),
            normalize,
        ])

    def __call__(self, image):
        crops = []
        crops.append(self.global_transfo1(image))
        crops.append(self.global_transfo2(image))
        for _ in range(self.local_crops_number):
            crops.append(self.local_transfo(image))
        return crops

##MultiCropWrapper

In [ ]:
class MultiCropWrapper(nn.Module):
    """
    This is essential in DINO-style self-supervised learning, where each image
    is transformed into several "views" (global and local crops) and all
    views must be processed efficiently.

    Args:
        backbone (nn.Module):  Feature extractor ( ResNet --> from pietro).
        head (nn.Module):      head (MLP) mapping extracted features to the output space.
    """
    def __init__(self, backbone: nn.Module = None, head:nn.Module=None):# : hint for the expected type
        super().__init__()
        # Store the backbone and head modules
        self.backbone = backbone
        self.head = head

    def forward(self, crops: List[Tensor]) -> List[Tensor]: #list of image crops (each crop is a tensor)
        """
        Performs a single forward pass for multiple crops of each image.

        -- Concatenate all crop tensors along the batch dimension.
          num of crops per image multiple the batch size to get the total batch size.

        -- Run the entire batch through the Mpdel to extract features.

        -- Flatten spatial dimensions if needed, resulting in (N*B, D_flat).

        -- Split the heading back into a list of N tensors, each of shape (B, out_dim).

        Args:
            crops (List[Tensor]):
                A list of image batches.
                The list length equals the number of crops per image.

        Returns:
            List[Tensor]:
                A list of head outputs (headings), each of shape (B, out_dim),
                in the same order as the input crops.
        """
        # Concatenate all crops into one large batch
        all_crops = torch.cat(crops, dim=0)

        # Feature extraction through thr model
        features = self.backbone(all_crops)

        # Flatten--->if needed
        if features.dim() > 2:
            features = torch.flatten(features, start_dim=1)

        # Project features to the dino head which is MLP
        heading = self.head(features)


        #  Split heading_inp back again
        #  divides the first dim into `len(crops)` equal parts of size B
        outputs = list(heading.chunk(len(crops), dim=0))

        return outputs




### Train

In [ ]:
def train_dino(args):
    print("I am a placeholder for the main training function.")
    # 1. init rand seed
    seed = 42
    torch.manual_seed(seed)
    np.random.seed(seed)

    # 2. prep data (build dataset, dataloader)
    dataset = datasets.ImageFolder(args.data_path, transform=DataAugmentationDINO(
            args.GLOBAL_CROPS_SCALE,
            args.LOCAL_CROPS_SCALE,
            args.LOCAL_CROPS_NUMBER,
        ))
    data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=args.BATCH_SIZE_PER_GPU,
        num_workers=args.num_works,
        pin_memory=True,
        drop_last=True,
    )

    # 3. build student, teacher, loss, optimizer
    # Backbone: ResNet50 as an example.
    backbone = models.resnet50(pretrained=False)
    backbone.fc = nn.Identity() # Replace the final FC layer with an identity layer
    in_dim = backbone.fc.in_features

    # Head for DINO
    head = DinoHead(
        in_dim,
        args.OUT_DIM,
        use_bn=args.USE_BN_IN_HEAD,
        norm_last_layer=args.NORM_LAST_LAYER,
    )

    student = MultiCropWrapper(backbone, head)
    teacher = MultiCropWrapper(models.resnet50(pretrained=False), DinoHead(
        in_dim,
        args.OUT_DIM,
        use_bn=args.USE_BN_IN_HEAD,
        norm_last_layer=args.NORM_LAST_LAYER,
    ))

    student = student.cuda()
    teacher = teacher.cuda()

    teacher.load_state_dict(student.state_dict())
    for p in teacher.parameters():
        p.requires_grad = False

    dino_loss = DINOLoss(args).cuda()

    optimizer = torch.optim.AdamW(student.parameters(), lr=args.LR, weight_decay=args.WEIGHT_DECAY)

    # 4. init schedulers
    #    This is where you would call the new function
    n_iter_per_epoch = len(data_loader)
    lr_schedule, wd_schedule, momentum_schedule = get_dino_schedules(args, n_iter_per_epoch)

    # 5. init loss_scaler
    loss_scaler = torch.cuda.amp.GradScaler(enabled=args.USE_FP16)

    # 6. train loop
    for epoch in range(args.EPOCHS):
        # Train one epoch
        train_stats = train_one_epoch(student, teacher, dino_loss, data_loader,
                                      optimizer, lr_schedule, wd_schedule, momentum_schedule,
                                      epoch, loss_scaler, args)

        print(f"Epoch {epoch} training stats: {train_stats}")

        # Save checkpoints
        if epoch % 10 == 0 or epoch == args.EPOCHS - 1:
            torch.save({
                'epoch': epoch,
                'student_state_dict': student.state_dict(),
                'teacher_state_dict': teacher.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss_scaler_state_dict': loss_scaler.state_dict(),
                'args': args,
            }, f'{args.output_dir}/checkpoint_{epoch:04d}.pth')
            print(f"Checkpoint saved to {args.output_dir}/checkpoint_{epoch:04d}.pth")

    print("DINO training finished!")

